In [1]:
from pyspark.sql import SparkSession
import os 
import sys 
import json 

os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = "notebook"
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_PYTHON'] = sys.executable

In [2]:
spark = (
    SparkSession
        .builder
        .appName('test')
        .getOrCreate()
    )

spark

In [3]:
poll_dir = "../data/poll_data.csv"

df = (
    spark.read.format("csv")
    # .option("encoding", "UTF-8")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(poll_dir)
)

df = (
    df.drop("Informazioni cronologiche")
    .drop(
        "Ho letto e accettato l'informativa e confermo inoltre di avere più di 18 anni"
    )
    .drop("Quanti anni hai?")
    .drop("Genere")
    .drop("Da quante persone è composto il tuo nucleo familiare?")
    .drop("Occupazione")
    .drop("Quanto è grande la tua azienda?")
    .drop("Da che regione provieni?")
    .drop("Provincia di provenienza")
    .drop("In che regione lavori/studi?")
    .drop("Provincia del luogo di lavoro/studio")
    .drop("Invalidità")
    .drop("Tipo di residenza")
    .drop("Numero di persone con cui convivi")
    .drop("Entrate Familiari Mensili Nette")
    .drop("Entrate Personali Mensili Nette ")
)

with open("../data/questions_ans_no_idxs.json", "r", encoding="utf-8") as questions_file:
    questions = json.load(questions_file)

questions_idxs = {questions[q]["question_text"]: q for q in questions.keys()}

for k, v in questions_idxs.items():
    df = df.withColumnRenamed(k, v)

df.show(5)

+---+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+----+--------------------+--------------------+--------------------+----+-------------------+--------------------+-------+--------------------+--------------------+---+---+------------------+--------------+---------------+----+--------------+------------+------------+--------------------+------------+----+--------------+-------------+--------------------+-------+----------------+--------------------+---------------+--------------------+-------+--------------------+----------------+--------------------+---------------+--------------------+--------------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+-----------+-------+-------+-------+-----------+-----------+-------+-----------+------+-----+-----------------+
|S_1|S_2|                 S_3|    

In [14]:
# print(questions_idxs)

scores = {}
for col in df.columns:
    question_text = questions[col]["question_text"]
    question_type = questions[col]["question_type"]
    question_score = questions[col]["question_score"]
    question_answers = questions[col]["answers"]

    # skip unscored questions
    if len(question_answers) == 0:
        continue

    # print("question: " + str(question_text))
    # print("question_type: " + str(question_type))
    # print("question_answers: " + str(question_answers))
    # print("question_score: " + str(question_score))

    answers = [str(d[0]) for d in df.select(col).collect()]
    # print(answers)

    row_scores = []
    if question_type in ["basic", "multivalue"]:
        for a in answers:
            row_scores.append(question_answers[a]["answer_score"] * question_score)

        # print("row_scores: " + str(row_scores))

        # col_scores = [answers[data[i][0]]["answer_score"] for i in range(len(data))]
        # scores.append(col_scores)
        scores[col] = row_scores

    elif question_type == "comma_separated":
        # print('implement multivalue questions handler')
        # ! temp
        scores[col] = [0 for i in range(52)]
        



In [17]:
for v in scores.values():
    print(len(v))

52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
1
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52
52


In [15]:
import pandas as pd

pd.DataFrame(scores)

ValueError: All arrays must be of the same length